In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import tensorflow as tf

In [19]:

import numpy as np
import pandas as pd
import math
import re
import nltk
nltk.download('all')
from nltk.corpus import wordnet as wn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from nltk.corpus import stopwords
#from ntlk.stem.porter import PorterStemmer





[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\kirby\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\kirby\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\kirby\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\kirby\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\kirby\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [83]:
full_dataset = pd.read_csv("reddit_opinion_climate_change.csv")
partial_dataset = full_dataset.get("self_text")

#VADER---------------------------------
analyzer = SentimentIntensityAnalyzer()
#---------------------------------------

#RobErta-------------------------------
ROBERTA_MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(ROBERTA_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(ROBERTA_MODEL)
#----------------------------------------



In [45]:
encoded_text = tokenizer("Hi", return_tensors='pt')



torch.Size([1, 3])

In [50]:
len("ha")

2

In [91]:
#full_dataset.loc[0].get("self_text")
fixed_roberta_partial = pd.Series()
fixed_partial = partial_dataset.dropna()
fixed_partial = fixed_partial.drop_duplicates()
fixed_partial = list(fixed_partial)
fixed_roberta_partial = fixed_partial  
fixed_partial = pd.Series(fixed_partial)
fixed_roberta_partial = pd.Series(fixed_roberta_partial)








        

        

    



In [92]:
type(fixed_roberta_partial)


pandas.core.series.Series

In [78]:
#Hee hee ha ha stolen from Vader
def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score/math.sqrt((score*score) + alpha)
    return norm_score

In [80]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    if(encoded_text["input_ids"].size(dim=1) > 512):
        return None
    else: 
        output = model(**encoded_text)
        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        scores_dict = {
            'neg' : -scores[0],
            'neu' : scores[1],
            'pos' : scores[2],
            'compound' : normalize(float(sum([scores[0], scores[2]])))
        }
        return scores_dict

{'neg': -0.4029202,
 'neu': 0.45570704,
 'pos': 0.14137273,
 'compound': 0.13916823662679345}

In [94]:
pos_score_series = []
neu_score_series = []
neg_score_series = []
comp_score_series = []
overall_sentiment_eval_series = []
model_used = []
next_sent_dict = None
next_pos_value = None
next_neu_value = None
next_neg_value = None
next_comp_value = None


#try:
for i in fixed_roberta_partial:
        
        next_sent_dict = polarity_scores_roberta(i)
        if(next_sent_dict == None):
                next_sent_dict = analyzer.polarity_scores(i)
                model_used.append("Vader")
        else:
                model_used.append("Roberta")

        next_pos_value = next_sent_dict["pos"]
        next_neu_value = next_sent_dict["neu"]
        next_neg_value = next_sent_dict["neg"]
        next_comp_value = next_sent_dict["compound"]
        pos_score_series.append(next_pos_value)
        neu_score_series.append(next_neu_value)
        neg_score_series.append(next_neg_value)
        comp_score_series.append(next_comp_value)
        
#except RuntimeError:
        #print(i + "/n")






KeyboardInterrupt: 

In [90]:
for j in comp_score_series:
        if(j < -0.1):
                overall_sentiment_eval_series.append("Negative")
        elif(j > 0.1):
                overall_sentiment_eval_series.append("Positive")
        else:
                overall_sentiment_eval_series.append("Neutral")


pos_score_series = pd.Series(pos_score_series)
neu_score_series = pd.Series(neu_score_series)
neg_score_series = pd.Series(neg_score_series)
comp_score_series = pd.Series(comp_score_series)
overall_sentiment_eval_series = pd.Series(overall_sentiment_eval_series)
print("list initialization complete!")

1190442

In [101]:

sentiment_text_dict = {"Reddit Comments": fixed_partial,
                       "Positive Scores": pos_score_series,
                       "Neutral Scores": neu_score_series,
                       "Negative Scores": neg_score_series,
                       "Composite Scores": comp_score_series,
                       "Overall Evaluation": overall_sentiment_eval_series
                       }
reddit_sentiment_dataframe = pd.DataFrame(sentiment_text_dict)
reddit_sentiment_dataframe = reddit_sentiment_dataframe.set_index("Reddit Comments")
reddit_sentiment_dataframe

,Positive Scores,Neutral Scores,Negative Scores,Composite Scores,Overall Evaluation
Reddit Comments,,,,,
Seeing parents letting their children look up without protection .. so bad...,0.000,1.000,0.000,0.0000,Neutral
You know who also benefits from WFH? The people that have to be on site because there is less traffic and completion for parking. \n\nPretty much everyone wins with WFH except commercial real estate.,0.235,0.765,0.000,0.8591,Positive
"Do you have any concept of how expensive public transportation is to operate and maintain? The only reason it is as cheap as it is is because it's heavily subsidized from tolls on drivers.\n\nIf driving was reduced, funding would be reduced but the price of tolls wouldn't go down (they virtually never do). So, to make up that lost revenue in the form of increased taxes, now the people who still need to drive have to still pay the same tolls AND now they get to pay a tax too. Then, the lower income people who never had to pay a tax on transit now have to pay fares and now a new tax.\n\nFor example, The NYC MTA is a black hole of money. Throwing more revenue at it never reduces costs to users.",0.019,0.894,0.087,-0.7579,Negative
"Renewables, namely hydro, were the primary mover for the first few decades of the Industrial Revolution. But the steam engine allowed for longer working hours and more control over the working class, so it won out despite the fuel being more expensive.\n\nRead *fossil capital* by Andreas Malm.",0.104,0.896,0.000,0.7556,Positive
??,0.000,1.000,0.000,0.0000,Neutral
...,...,...,...,...,...
"Yes, I remember the hype that ended in nothing much, but OP has a point: if the government would do this to reduce pollution and improve health, profit wouldn't be the key driver. China is pushing for massive changes to improve the air quality, making fuel could be an added bonus...however there must be other drawbacks/ hurdles...I could imagine space being one in major cities. I have no idea how much algae you would need to make a significant difference but I would imagine it would be a substantial amount that would be difficult to accommodate for on eg rooftops. And while CO2 is a major problem, the reason is the fact that it is a greenhouse gas. it isn't what causes the immediate respiratory health effects in cities. The key air pollutants are NOx (nitrogen oxides),SOx (sulphur oxides)-and PMs (particulate matter) and O3.\nWhile plants can filter some NOx out, I don't think (but anyone that knows better, please jump in!) They would be able to reduce SOx and especially PMs from the air. \n\nWithout that benefit your back to sequestration (which you can do elsewhere more easily and is not a longterm solution anyway) and possibly quite expensive biofuel... These are not likely to be motivations that will ensure a government involvement.",0.135,0.800,0.065,0.9483,Positive
"There were a [lot of people who lost a lot of money trying to do this 5-8 years ago](https://www.greentechmedia.com/articles/read/lessons-from-the-great-algae-biofuel-bubble). It's not something that's impossible, but it is something nobody has figured out how to make money doing.",0.000,0.954,0.046,-0.1655,Negative
"Well, it can be used as a bio fuel. This is an easy solution to a huge problem. Profit shouldn't matter here. The government could tax fuel and grow algae with it. Hell if it worked that well if do it in my back garden!\n",0.260,0.594,0.146,0.6724,Positive
